In [1]:
!pip install -q accelerate transformers==4.28.0 
!pip install -q datasets==2.12.0 
!pip install -q evaluate shortuuid gradio visen

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import warnings

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict, Dataset

In [3]:
# load squad v2
squad = load_dataset('squad_v2')
temp = squad['train'].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_origin = temp['train']
val_origin = temp['test']
test_origin = squad['validation']

train_origin, val_origin, test_origin

Found cached dataset squad_v2 (/home/jupyter/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/jupyter/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-8392fe7aa8289e19.arrow and /home/jupyter/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-10963ca112a1f503.arrow


(Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 117287
 }),
 Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 13032
 }),
 Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 11873
 }))

In [4]:
model_checkpoint = 'roberta-base'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [28]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.28.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [5]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    example_ids = []
    for i in range(len(inputs["input_ids"])):
        example_ids.append(examples["id"][i])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


In [6]:
train_dataset = train_origin.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_origin.column_names,
)
train_dataset

Map:   0%|          | 0/117287 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 117287
})

In [7]:
val_dataset = val_origin.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=val_origin.column_names,
)
val_dataset

Map:   0%|          | 0/13032 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 13032
})

In [8]:
test_dataset = test_origin.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_origin.column_names,
)
test_dataset

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 11873
})

In [20]:
check_point = "viettel-roberta-finetune-squad2-baseline"
args = TrainingArguments(
    check_point,
    save_strategy="epoch",
    evaluation_strategy='steps',
    eval_steps=1100,
    learning_rate=2e-5,
    num_train_epochs=2,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    weight_decay=0.01,
    gradient_accumulation_steps=3,
    warmup_steps=100,
    # push_to_hub=True,
    overwrite_output_dir=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [21]:
trainer.train()

Step,Training Loss,Validation Loss
1100,0.875300,0.863667
2200,0.821600,0.837895
3300,0.671300,0.801203


TrainOutput(global_step=3910, training_loss=0.7665554349074888, metrics={'train_runtime': 5374.8239, 'train_samples_per_second': 43.643, 'train_steps_per_second': 0.727, 'total_flos': 6.129342541459046e+16, 'train_loss': 0.7665554349074888, 'epoch': 2.0})

In [22]:
trainer.push_to_hub()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.push_to_hub()                                                                        │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3639 in push_to_hub              │
│                                                                                                  │
│   3636 │   │   # If a user calls manually `push_to_hub` with `self.args.push_to_hub = False`, w  │
│   3637 │   │   # it might fail.                                                                  │
│   3638 │   │   if not hasattr(self, "repo"):                                                     │
│ ❱ 3639 │   │   │   self.init_git_repo()                                                          │
│   3640 │   │                                                                                     │
│   3641 │   │   model_name = kwargs.pop("model_name", None)                                       │
│   3642 │   │   if model_name is None and self.args.should_save:                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3492 in init_git_repo            │
│                                                                                                  │
│   3489 │   │   else:                                                                             │
│   3490 │   │   │   repo_name = self.args.hub_model_id                                            │
│   3491 │   │   if "/" not in repo_name:                                                          │
│ ❱ 3492 │   │   │   repo_name = get_full_repo_name(repo_name, token=self.args.hub_token)          │
│   3493 │   │                                                                                     │
│   3494 │   │   # Make sure the repo exists.                                                      │
│   3495 │   │   create_repo(repo_name, token=self.args.hub_token, private=self.args.hub_private_  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:795 in get_full_repo_name      │
│                                                                                                  │
│    792                                                                                           │
│    793 def get_full_repo_name(model_id: str, organization: Optional[str] = None, token: Optiona  │
│    794 │   if organization is None:                                                              │
│ ❱  795 │   │   username = whoami(token)["name"]                                                  │
│    796 │   │   return f"{username}/{model_id}"                                                   │
│    797 │   else:                                                                                 │
│    798 │   │   return f"{organization}/{model_id}"                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_toke

In [26]:
from collections import defaultdict
import evaluate
from tqdm.auto import tqdm
metric = evaluate.load("squad_v2")
from tqdm.auto import tqdm
def compute_metrics(
    start_logits,
    end_logits,
    features,
    examples,
    metric=metric,
    n_best=10,
    max_answer_length=100,
    show_result=True,
):
    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            candidate_answers = []
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip candidate_answers that are not fully in the context
                    if start_index ==0 and end_index==0:
                        candidate_answers.append({
                            'text':"",
                            'logit_score': start_logit[start_index] + end_logit[end_index],
                        })
                        continue
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip candidate_answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    answer = {
                        "text": context[
                            offsets[start_index][0] : offsets[end_index][1]
                        ],
                        "logit_score": start_logit[start_index] + end_logit[end_index],

                    }
                    candidate_answers.append(answer)

            # Add the best candidate_answers with a score > zero to the final list
            candidate_answers = sorted(
                candidate_answers, key=lambda x: x["logit_score"], reverse=True
            )
            if len(candidate_answers) > 0:
                answers.append(candidate_answers[0])
        if len(answers) == 0:
            answers.append({"text": "", "logit_score": 0.0})
            
        best_answer = max(answers, key=lambda x: x["logit_score"])
        if best_answer['text'] == "":
            predicted_answers.append(
                {"id": example_id, "prediction_text": "", "no_answer_probability": 1.0}
            )
        else:
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"], "no_answer_probability": 0.0}
            )

    theoretical_answers = [
        {
            "id": ex["id"],
            "answers": {
                "text": ex["answers"]["text"],
                "answer_start": ex["answers"]["answer_start"],
            },
        }
        for ex in examples
    ]
    if show_result:
        i = 0
        for pred, theo in zip(predicted_answers, theoretical_answers):
            i += 1
            if i == 100:
                break
            print(pred['prediction_text'])
            print(theo['answers']['text'])
            print('----' * 30)
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [24]:
predicitions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predicitions

In [27]:
compute_metrics(start_logits, end_logits, test_dataset, test_origin, show_result=False)

  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 79.4660153288975,
 'f1': 82.64010615108855,
 'total': 11873,
 'HasAns_exact': 78.39068825910931,
 'HasAns_f1': 84.74797239066709,
 'HasAns_total': 5928,
 'NoAns_exact': 80.53826745164004,
 'NoAns_f1': 80.53826745164004,
 'NoAns_total': 5945,
 'best_exact': 79.4660153288975,
 'best_exact_thresh': 0.0,
 'best_f1': 82.64010615108829,
 'best_f1_thresh': 0.0}